# package installation & load data


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objects as go

In [ ]:
df = pd.read_csv('.../portfolio_data.csv')

# Data cleaning

In [ ]:
df[['Cycle Number', 'Year']] = df['Cycle'].str.split(' ', expand=True)[[1, 2]] #split cycle and year

In [ ]:
df[['First name', 'Last name']] = df['Project Owner'].str.split(' ', expand=True)[[0, 1]] #split first name and last name

In [ ]:
df[['Primary area', 'Secondary area']] = df['Programmatic Priority'].str.split(';', expand=True)[[0, 1]]

In [ ]:
df['Year'] = pd.to_numeric(df['Year'], errors='coerce')

In [ ]:
df.rename(columns={'Short Project Title': 'Project Title'}, inplace=True)

In [ ]:
selected_columns = ['Project Title', 'Technical Abstract', 'Methods used']

for col_name in selected_columns:
    df[col_name] = df[col_name].apply(lambda x: f"{col_name}: {x}")

In [ ]:
# Combining content from selected columns into a new column 'Project Content'
df['Project Content'] = df.apply(lambda row: ' '.join(str(row[col]) for col in selected_columns), axis=1)

# Apply keyBert/KeyLLM for keyword extraction

In [ ]:
!pip install keybert
!pip install transformers
!pip install bitsandbytes
!pip install accelerate

In [ ]:
import bitsandbytes
from torch import cuda, bfloat16
import transformers
from huggingface_hub.hf_api import HfFolder
import accelerate

from keybert.llm import TextGeneration
from keybert import KeyLLM, KeyBERT

In [ ]:
# 4-bit Quantization to load Llama 2 with less GPU memory
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

In [ ]:
model_id = 'meta-llama/Llama-2-7b-chat-hf'
HfFolder.save_token("hf_xxxx")

In [ ]:
tokenizer = transformers.AutoTokenizer.from_pretrained(model_id)
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map='auto',
)
model.eval()

In [ ]:
# Our text generator
generator = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    task='text-generation',
    temperature=0.1,
    max_new_tokens=500,
    repetition_penalty=1.1
)

In [ ]:
prompt = """
<s>[INST] <<SYS>>

You are a helpful assistant specialized in extracting comma-separated keywords.
Please identify important topics, for example clinical condition, research objective, analytical methods used, research gap addressed from
the description of a research proposal. Please read and comprehend the description, and identify suitable terms to annotate that proposal, so
that by looking at just the topical terms, reader can get a general sense of what this proposal is about.

<</SYS>>
I have the following project description:
Project Title: [input project title here].
[input project abstract here]
Our Specific Aims are to:[input specific aims here].
Methods used: [input methods used here]

Please give me the keywords that are present in this document and separate them with commas.
The keywords can be more than one word. For example, don't say:
"Here are the keywords present in the document"
[/INST] observational comparative effectiveness research, longitudinal phenotypes extraction, real-world data, electronic health records, confounder ascertainment, patient-reported outcomes, machine learning,
generalizability, natural language processing, propensity scores, cancer research [INST]

I have the following document:
- [DOCUMENT]

Please give me the keywords that are present in this document and separate them with commas.
Make sure you to only return the keywords and say nothing else. For example, don't say:
"Here are the keywords present in the document"
[/INST]
"""

# Load it in KeyLLM
llm = TextGeneration(generator, prompt=prompt)
kw_model = KeyLLM(llm)
#kw_model = KeyBERT(llm=llm)

In [ ]:
# let's look at one example output
keywords = kw_model.extract_keywords(text_51,check_vocab=True)
keywords

In [ ]:
extracted_keywords = []

for index, row in df.iterrows():
    # Extract keywords for each 'Project Content' value using kw_model.extract_keywords
    keywords = kw_model.extract_keywords(row['Project Content'], check_vocab=True)

    # Append extracted keywords to the list
    extracted_keywords.append(keywords)

# Test KeyBERT

In [ ]:
extracted_keywords_1 = []
kw_model_2 = KeyBERT(llm=llm)

for index, row in subset_df.iterrows():
    # Extract keywords for each 'Project Content' value using kw_model.extract_keywords
    keywords = kw_model_2.extract_keywords(row['Project Content'], keyphrase_ngram_range=(1, 2), stop_words=None)

    # Append extracted keywords to the list
    extracted_keywords_1.append(keywords)

In [ ]:
subset_df['Extracted Keywords_KeyLLM'] = extracted_keywords_1